In [2]:
import pandas as pd
from tqdm import tqdm

In [3]:
# load data
rating_data = pd.read_csv('data/rating_BETA_Jan.csv', encoding='utf-8-sig')
user_data = pd.read_csv('data/user_feature_BETA_Jan.csv', encoding='utf-8-sig')
mat_data = pd.read_csv('data/material_feature_BETA_Jan.csv', encoding='utf-8-sig')
review_data = pd.read_csv('data/review_BETA_Jan.csv', encoding='utf-8-sig')

In [4]:
# only materialpointsCNT == 1 in rating_data
# w/o repeat mat score in rating_data
rating_data = rating_data[rating_data['materialpointsCNT'] == 1]
uid_list = list(set(rating_data['client_sn']))
rating_data_wo_repeat = list()
for uid in tqdm(uid_list):
    dat = rating_data[rating_data['client_sn'] == uid]
    if len(set(dat['material_points'])) > 1:
        rating_data_wo_repeat.append(dat)
rating_data = pd.concat(rating_data_wo_repeat).reset_index(drop=True) 

100%|██████████| 36787/36787 [00:34<00:00, 1073.63it/s]


In [5]:
# add label feature to rating_data
import numpy as np
rating_data['label'] = [np.nan for _ in range(rating_data.shape[0])]
uid_list = list(set(rating_data['client_sn']))
for uid in tqdm(uid_list):
    dat = rating_data[rating_data['client_sn'] == uid]
    index = dat.index
    score_list = list(dat['material_points'])
    max_score = max(score_list)
    label_list = []
    for score in score_list:
        if score == max_score:
            label_list.append(1)
        else:
            label_list.append(0)
    rating_data.loc[index, 'label']   = label_list

100%|██████████| 19086/19086 [00:44<00:00, 431.95it/s]


In [6]:
# select 'client_sn','MaterialID','session_sn','PurchaseBrandID','attend_level','attend_date','label' as feature in rating_data
rating_data = rating_data[['client_sn','MaterialID','session_sn','PurchaseBrandID','attend_level','attend_date','label']]

In [7]:
# build rating_review_data by merging rating_data, review_data. where key = ['client_sn','MaterialID','session_sn'] (left join)
rating_review_data = pd.merge(rating_data, review_data, on=['client_sn','MaterialID','session_sn'], how='left')

In [8]:
def transform_date_to_age(date_str, categorical=True):
    if date_str != 'None':
        age_val = 2021 - pd.to_datetime(date_str, format='%Y-%m-%d %H:%M:%S').year
        if categorical is False:
            return age_val
        else:
            if age_val <= 30:
                return '0~30'
            elif age_val > 30 and age_val < 50:
                return '30~50'
            else:
                return '50~'   
    else:
        return 'None'

In [9]:
user_data = user_data.fillna('None')
user_data['Client_Sex'].replace('N','None')
user_data['birthday'] = user_data['birthday'].apply(lambda x: transform_date_to_age(x))
user_data['JobClassName'].replace('Undefined','None')
user_data['IndustryClassName'].replace('Undefined','None')
user_data = user_data[['client_sn','Client_Sex','birthday','education','JobClassName','IndustryClassName']]

In [10]:
rating_review_data_with_UF = pd.merge(rating_review_data, user_data, on=['client_sn'], how='left')

In [11]:
rating_data_with_UF = pd.merge(rating_data, user_data, on=['client_sn'], how='left')

In [12]:
mat_individual_col = list(set(rating_review_data_with_UF.columns)-{'client_sn','MaterialID','session_sn','PurchaseBrandID','attend_level','material_points','con_sn','label','attend_date','Client_Sex','birthday','education','JobClassName','IndustryClassName'})
mat_individual_dat = rating_review_data_with_UF.groupby(['MaterialID','Client_Sex','birthday','education','JobClassName','IndustryClassName']).mean()[mat_individual_col]

In [15]:
df = rating_review_data_with_UF.groupby(['MaterialID']).mean()[mat_individual_col]

In [16]:
df.head(10)

,complaint_ECA,complaint_ICA,complaint_ECV,complaint_EAG,complaint_DFV,M_Point,complaint_ECR,count_overall_points,complaint_ICV,T_Point,...,compliment_COR,C_Point,complaint_EAV,complaint_BOR,complaint_OFA,compliment_INT,complaint_DFG,count_consultant_points,complaint_ICR,count_materials_points
MaterialID,,,,,,,,,,,,,,,,,,,,,
100059,0.000000,0.030612,0.000000,0.030612,0.010204,9.428571,0.020408,1.0,0.040816,9.612245,...,0.173469,9.693878,0.020408,0.040816,0.010204,0.540816,0.000000,1.0,0.040816,1.0
100060,0.018750,0.006250,0.021875,0.003125,0.081250,9.178125,0.053125,1.0,0.009375,9.234375,...,0.287500,9.262500,0.003125,0.084375,0.015625,0.528125,0.096875,1.0,0.015625,1.0
100064,0.000000,0.056604,0.000000,0.018868,0.000000,9.377358,0.000000,1.0,0.018868,9.339623,...,0.283019,9.509434,0.018868,0.075472,0.000000,0.471698,0.000000,1.0,0.018868,1.0
100180,0.000000,0.013699,0.000000,0.041096,0.000000,9.452055,0.000000,1.0,0.027397,9.383562,...,0.287671,9.630137,0.041096,0.013699,0.000000,0.287671,0.000000,1.0,0.054795,1.0
100183,0.000000,0.018987,0.000000,0.012658,0.000000,9.411392,0.006329,1.0,0.012658,9.417722,...,0.196203,9.626582,0.031646,0.025316,0.000000,0.373418,0.006329,1.0,0.018987,1.0
100219,0.000000,0.000000,0.000000,0.000000,0.000000,9.558824,0.000000,1.0,0.000000,9.441176,...,0.323529,9.205882,0.000000,0.029412,0.000000,0.529412,0.029412,1.0,0.000000,1.0
100259,0.000000,0.058824,0.000000,0.058824,0.000000,9.220588,0.000000,1.0,0.044118,9.294118,...,0.102941,9.455882,0.132353,0.044118,0.000000,0.426471,0.000000,1.0,0.088235,1.0
100261,0.007519,0.000000,0.000000,0.015038,0.000000,9.556391,0.007519,1.0,0.007519,9.533835,...,0.225564,9.548872,0.015038,0.022556,0.007519,0.263158,0.030075,1.0,0.000000,1.0
100262,0.000000,0.000000,0.000000,0.000000,0.000000,9.487179,0.000000,1.0,0.017094,9.589744,...,0.205128,9.606838,0.008547,0.008547,0.000000,0.316239,0.025641,1.0,0.008547,1.0


In [19]:
mat_individual_dat.groupby(['MaterialID']).mean()

,complaint_ECA,complaint_ICA,complaint_ECV,complaint_EAG,complaint_DFV,M_Point,complaint_ECR,count_overall_points,complaint_ICV,T_Point,...,compliment_COR,C_Point,complaint_EAV,complaint_BOR,complaint_OFA,compliment_INT,complaint_DFG,count_consultant_points,complaint_ICR,count_materials_points
MaterialID,,,,,,,,,,,,,,,,,,,,,
100059,0.000000,0.054203,0.000000,0.054203,0.000870,9.526615,0.001739,1.0,0.055072,9.539658,...,0.236646,9.683778,0.053333,0.055072,0.000870,0.556263,0.000000,1.0,0.094203,1.0
100060,0.010114,0.002743,0.005347,0.000521,0.159176,9.256771,0.027749,1.0,0.001563,9.530537,...,0.179943,9.518131,0.000521,0.032689,0.002604,0.425303,0.046057,1.0,0.002604,1.0
100064,0.000000,0.017241,0.000000,0.005747,0.000000,9.370626,0.000000,1.0,0.005747,9.384483,...,0.258940,9.663218,0.005747,0.035760,0.000000,0.367305,0.000000,1.0,0.005747,1.0
100180,0.000000,0.001818,0.000000,0.010303,0.000000,9.522020,0.000000,1.0,0.015152,9.425455,...,0.197172,9.652929,0.005455,0.004444,0.000000,0.219798,0.000000,1.0,0.012121,1.0
100183,0.000000,0.029677,0.000000,0.006786,0.000000,9.439899,0.001786,1.0,0.001701,9.545386,...,0.278930,9.668006,0.031531,0.003401,0.000000,0.384920,0.001786,1.0,0.015094,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1211901,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000,1.0,0.000000,8.000000,...,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,1.0
1211907,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000,1.0,0.000000,7.000000,...,0.000000,8.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,1.0
1211917,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,1.0,0.000000,9.000000,...,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,1.0


In [13]:
mat_individual_dat.shape

(118810, 21)

In [16]:
mat_individual_dat.head(30)

complaint_EAV  \
MaterialID Client_Sex birthday education    JobClassName  IndustryClassName                                    
100059     F          0~30     None         Undefined     Undefined                                 0.500000   
                      30~50    None         General Staff Marketing and Sales                       0.000000   
                                            Undefined     Undefined                                 0.000000   
                               other-code:0 Undefined     Undefined                                 0.000000   
                               大专/**/專科/大學  General Staff Manufacturing and retailing               0.000000   
                                            Management    Professional and technical services       0.000000   
                               硕士/**/碩/博士   Management    Professional and technical services       0.000000   
                      50~      None         Undefined     Undefined                                 0.000000   
                               other-code:0 Undefined     Undefined                                 0.000000   
                               中专/**/高中/職   Undefined     Undefined                                 0.000000   
                               大专/**/專科/大學  Undefined     Undefined                                 0.000000   
           M          0~30     None         Engineer      Student                                   0.000000   
                                            Undefined     Undefined                                 0.000000   
                               other-code:0 Undefined     Undefined                                 0.000000   
                      30~50    None         Undefined     Undefined                                 0.000000   
                               大专/**/專科/大學  Undefined     Undefined                                 0.000000   
                      50~      None         Professional  Information                               0.000000   
                                            Undefined     Undefined                                 0.000000   
                               other-code:0 Undefined     Undefined                                 0.000000   
           N          0~30     None         Undefined     Undefined                                 0.000000   
           None       None     None         None          None                                      0.000000   
100060     F          0~30     None         Undefined     Undefined                                 0.000000   
                               other-code:0 Undefined     Undefined                                 0.000000   
                               大专/**/專科/大學  Undefined     Undefined                                 0.000000   
                      30~50    None         Professional  Transportation and trading                0.000000   
           M          0~30     None         Undefined     Undefined                                 0.000000   
                               other-code:0 Undefined     Undefined                                 0.000000   
           N          0~30     None         Undefined     Undefined                                 0.000000   
           None       None     None         None          None                                      0.005952   
100064     F          0~30     None         Undefined     Undefined                                 0.000000   

                                                                                               complaint_EAG  \
MaterialID Client_Sex birthday education    JobClassName  IndustryClassName                                    
100059     F          0~30     None         Undefined     Undefined                                 0.500000   
                      30~50    None         General Staff Marketing and Sales                       0.000000   
                                            Undefined     Unde

In [17]:
# build rating_matF_data by merging rating_data, mat_individual_dat
rating_matF_data = pd.merge(rating_data_with_UF, mat_individual_dat, on=['MaterialID','Client_Sex','birthday','education','JobClassName','IndustryClassName'], how='left')

In [18]:
# sperate adult, jr data by PurchaseBrandID
rating_matF_data_AD = rating_matF_data[rating_matF_data['PurchaseBrandID']==1]
rating_matF_data_Jr = rating_matF_data[rating_matF_data['PurchaseBrandID']!=1]
rating_matF_data_AD = rating_matF_data_AD[list(set(rating_matF_data_AD.columns)-{'session_sn','PurchaseBrandID','Client_Sex','birthday','education','JobClassName','IndustryClassName'})]
rating_matF_data_Jr = rating_matF_data_Jr[list(set(rating_matF_data_Jr.columns)-{'session_sn','PurchaseBrandID','Client_Sex','birthday','education','JobClassName','IndustryClassName'})]

In [19]:
# sperate train, test data by attend_date
start_date = '2021-01-01'
train_date = '2021-04-01'
end_date = '2021-05-01'
train_data_AD = rating_matF_data_AD[(rating_matF_data_AD['attend_date'] >= start_date) & (rating_matF_data_AD['attend_date'] < train_date)]
test_data_AD = rating_matF_data_AD[(rating_matF_data_AD['attend_date'] >= train_date) & (rating_matF_data_AD['attend_date'] < end_date)]
train_data_Jr = rating_matF_data_Jr[(rating_matF_data_Jr['attend_date'] >= start_date) & (rating_matF_data_Jr['attend_date'] < train_date)]
test_data_Jr = rating_matF_data_Jr[(rating_matF_data_Jr['attend_date'] >= train_date) & (rating_matF_data_Jr['attend_date'] < end_date)]

In [20]:
train_data_AD = train_data_AD[list(set(train_data_AD.columns)-{'attend_date'})]
test_data_AD = test_data_AD[list(set(test_data_AD.columns)-{'attend_date'})]
train_data_Jr = train_data_Jr[list(set(train_data_Jr.columns)-{'attend_date'})]
test_data_Jr = test_data_Jr[list(set(test_data_Jr.columns)-{'attend_date'})]

In [21]:
train_data_AD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93158 entries, 0 to 325981
Data columns (total 19 columns):
complaint_EAG     93156 non-null float64
label             93158 non-null float64
complaint_DFG     93156 non-null float64
complaint_ICV     93156 non-null float64
complaint_ECV     93156 non-null float64
complaint_ECR     93156 non-null float64
MaterialID        93158 non-null int64
complaint_EAV     93156 non-null float64
compliment_INT    93156 non-null float64
attend_level      93158 non-null int64
compliment_COR    93156 non-null float64
compliment_PRA    93156 non-null float64
complaint_ICR     93156 non-null float64
client_sn         93158 non-null int64
complaint_ICA     93156 non-null float64
complaint_BOR     93156 non-null float64
complaint_OFA     93156 non-null float64
complaint_ECA     93156 non-null float64
complaint_DFV     93156 non-null float64
dtypes: float64(16), int64(3)
memory usage: 14.2 MB


In [22]:
test_data_AD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32146 entries, 13 to 325996
Data columns (total 19 columns):
complaint_EAG     32146 non-null float64
label             32146 non-null float64
complaint_DFG     32146 non-null float64
complaint_ICV     32146 non-null float64
complaint_ECV     32146 non-null float64
complaint_ECR     32146 non-null float64
MaterialID        32146 non-null int64
complaint_EAV     32146 non-null float64
compliment_INT    32146 non-null float64
attend_level      32146 non-null int64
compliment_COR    32146 non-null float64
compliment_PRA    32146 non-null float64
complaint_ICR     32146 non-null float64
client_sn         32146 non-null int64
complaint_ICA     32146 non-null float64
complaint_BOR     32146 non-null float64
complaint_OFA     32146 non-null float64
complaint_ECA     32146 non-null float64
complaint_DFV     32146 non-null float64
dtypes: float64(16), int64(3)
memory usage: 4.9 MB


In [23]:
train_data_Jr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117187 entries, 62 to 326201
Data columns (total 19 columns):
complaint_EAG     117187 non-null float64
label             117187 non-null float64
complaint_DFG     117187 non-null float64
complaint_ICV     117187 non-null float64
complaint_ECV     117187 non-null float64
complaint_ECR     117187 non-null float64
MaterialID        117187 non-null int64
complaint_EAV     117187 non-null float64
compliment_INT    117187 non-null float64
attend_level      117187 non-null int64
compliment_COR    117187 non-null float64
compliment_PRA    117187 non-null float64
complaint_ICR     117187 non-null float64
client_sn         117187 non-null int64
complaint_ICA     117187 non-null float64
complaint_BOR     117187 non-null float64
complaint_OFA     117187 non-null float64
complaint_ECA     117187 non-null float64
complaint_DFV     117187 non-null float64
dtypes: float64(16), int64(3)
memory usage: 17.9 MB


In [24]:
test_data_Jr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35933 entries, 66 to 326205
Data columns (total 19 columns):
complaint_EAG     35933 non-null float64
label             35933 non-null float64
complaint_DFG     35933 non-null float64
complaint_ICV     35933 non-null float64
complaint_ECV     35933 non-null float64
complaint_ECR     35933 non-null float64
MaterialID        35933 non-null int64
complaint_EAV     35933 non-null float64
compliment_INT    35933 non-null float64
attend_level      35933 non-null int64
compliment_COR    35933 non-null float64
compliment_PRA    35933 non-null float64
complaint_ICR     35933 non-null float64
client_sn         35933 non-null int64
complaint_ICA     35933 non-null float64
complaint_BOR     35933 non-null float64
complaint_OFA     35933 non-null float64
complaint_ECA     35933 non-null float64
complaint_DFV     35933 non-null float64
dtypes: float64(16), int64(3)
memory usage: 5.5 MB


In [25]:
train_data_AD = train_data_AD.dropna()
test_data_AD = test_data_AD.dropna()
train_data_Jr = train_data_Jr.dropna()
test_data_Jr = test_data_Jr.dropna()

In [26]:
label_AD = np.array(train_data_AD['label'])
label_Jr = np.array(train_data_Jr['label'])
train_data_AD = train_data_AD[list(set(train_data_AD.columns)-{'label'})]
train_data_Jr = train_data_Jr[list(set(train_data_Jr.columns)-{'label'})]
ground_truth_AD = np.array(test_data_AD['label'])
ground_truth_Jr = np.array(test_data_Jr['label'])
test_data_AD = test_data_AD[list(set(test_data_AD.columns)-{'label'})]
test_data_Jr = test_data_Jr[list(set(test_data_Jr.columns)-{'label'})]

In [27]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
def train_model(train_data,label):
    train_data = np.array(train_data)
    rf = RandomForestRegressor()
    rf.fit(train_data, label)
    return rf

In [28]:
model_AD = train_model(train_data=train_data_AD, label=label_AD)
model_Jr = train_model(train_data=train_data_Jr, label=label_Jr)

/home/hsucheng/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/hsucheng/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [29]:
import sklearn.metrics as metrics 
from sklearn.metrics import confusion_matrix
def predict_score(pred_prob, Y_test_array, binary_threshold=0.5):
    pred_one_hot = list()
    for i in range(pred_prob.shape[0]):
        if pred_prob[i] >= binary_threshold:
            pred_one_hot.append(1)
        else:
            pred_one_hot.append(0)
    print(metrics.classification_report(list(Y_test_array), pred_one_hot))
    print('---------------------------------------')
    print('Confusion Matrix')
    print(np.transpose(confusion_matrix(list(Y_test_array), pred_one_hot).T))
    print('---------------------------------------')
    print('positive label : 1 | negative label : 0')

In [30]:
test_data_AD_array = np.array(test_data_AD)
y_pred_AD = model_AD.predict(test_data_AD_array)
predict_score(y_pred_AD,ground_truth_AD)

              precision    recall  f1-score   support

         0.0       0.58      0.45      0.51     11923
         1.0       0.71      0.80      0.76     20223

    accuracy                           0.67     32146
   macro avg       0.64      0.63      0.63     32146
weighted avg       0.66      0.67      0.66     32146

---------------------------------------
Confusion Matrix
[[ 5379  6544]
 [ 3972 16251]]
---------------------------------------
positive label : 1 | negative label : 0


In [31]:
ground_truth_AD_list = list(ground_truth_AD)
pos,neg = 0,0
for val in ground_truth_AD_list:
    if int(val) == 1:
        pos +=1
    else:
        neg +=1
import random
random_pred = np.array([random.sample([1,0],1)[0] for _ in range(len(ground_truth_AD))])
import sklearn.metrics as metrics 
from sklearn.metrics import confusion_matrix
predict_score(random_pred,ground_truth_AD_list)

              precision    recall  f1-score   support

         0.0       0.37      0.50      0.43     11923
         1.0       0.63      0.51      0.56     20223

    accuracy                           0.50     32146
   macro avg       0.50      0.50      0.50     32146
weighted avg       0.54      0.50      0.51     32146

---------------------------------------
Confusion Matrix
[[ 5992  5931]
 [10009 10214]]
---------------------------------------
positive label : 1 | negative label : 0


In [32]:
test_data_Jr_array = np.array(test_data_Jr)
y_pred_Jr = model_Jr.predict(test_data_Jr_array)
predict_score(y_pred_Jr,ground_truth_Jr)

              precision    recall  f1-score   support

         0.0       0.42      0.30      0.35     12373
         1.0       0.68      0.78      0.73     23560

    accuracy                           0.61     35933
   macro avg       0.55      0.54      0.54     35933
weighted avg       0.59      0.61      0.60     35933

---------------------------------------
Confusion Matrix
[[ 3651  8722]
 [ 5119 18441]]
---------------------------------------
positive label : 1 | negative label : 0


In [33]:
ground_truth_Jr_list = list(ground_truth_Jr)
pos,neg = 0,0
for val in ground_truth_Jr_list:
    if int(val) == 1:
        pos +=1
    else:
        neg +=1
import random
random_pred = np.array([random.sample([1,0],1)[0] for _ in range(len(ground_truth_Jr_list))])
import sklearn.metrics as metrics 
from sklearn.metrics import confusion_matrix
predict_score(random_pred,ground_truth_Jr_list)

              precision    recall  f1-score   support

         0.0       0.34      0.50      0.41     12373
         1.0       0.66      0.50      0.57     23560

    accuracy                           0.50     35933
   macro avg       0.50      0.50      0.49     35933
weighted avg       0.55      0.50      0.51     35933

---------------------------------------
Confusion Matrix
[[ 6195  6178]
 [11801 11759]]
---------------------------------------
positive label : 1 | negative label : 0
